In [72]:
from tensorflow.keras.models import model_from_json

# Model reconstruction from JSON file
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('weights.best.h5')

In [73]:
import os 

original_dataset_dir = './dataset/dogs_and_cats'
test_dir = os.path.join(original_dataset_dir, "test")
filenames = [filename for filename in os.listdir(test_dir)]

filenames[:5]

['4224.jpg', '9426.jpg', '4955.jpg', '9847.jpg', '9993.jpg']

In [74]:
len(filenames)

12500

In [75]:
import pandas as pd

data = {"filename": filenames, "label": [0]*len(filenames)}

test_df = pd.DataFrame(data)

In [76]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_generator = ImageDataGenerator(
    rescale=1./255)


test_datagenerator = image_generator.flow_from_dataframe(
  test_df, test_dir, y_col=None, target_size=(150,150),batch_size=500, class_mode=None)

Found 12500 images.


In [77]:
y_probabilities = model.predict_generator(test_datagenerator)

In [78]:
y_probabilities

array([[0.9912578 ],
       [0.97223264],
       [0.09664908],
       ...,
       [0.647135  ],
       [0.12673998],
       [0.9748258 ]], dtype=float32)

In [79]:
pred = (y_probabilities > 0.5).flatten() + 0

In [80]:
data = {"id": [filename.split(".")[0] for filename in filenames], "label": y_probabilities.flatten()}

result_df = pd.DataFrame(data)

In [81]:
result_df.set_index("id").to_csv("submission.csv")

In [82]:
!kaggle competitions submit -f submission.csv -m "First Submission" dogs-vs-cats-redux-kernels-edition

100%|████████████████████████████████████████| 195k/195k [00:04<00:00, 48.6kB/s]
Successfully submitted to Dogs vs. Cats Redux: Kernels Edition